In [94]:
import tweepy
import json
import jsonpickle
import numpy
import pandas as pd
import time
import datetime

In [2]:
auth = tweepy.OAuthHandler('vaHi8ttuBNsl27RXTBaEW7LFw', '6tkC5bMapFnm8Z69ae8pqB6N12x7H0aoI805kyllVkSwAJOdTC')
auth.set_access_token('34403159-dyGSWk0uvdSU0BSPwwcYlPqCFUKVsiddkRQGg8pt6','8FbZkNgQZqTnd3NKHHkm7WETFSEpf7w2PSShlHT3Nnl0y')

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [3]:
#Shows my timeline
public_tweets = api.home_timeline()
for tw in public_tweets:
    print(tw.text)

RT @gwenckatz: Lookie lookie, @DanKoboldt! https://t.co/mUxplM0l2w
Want to work at Actus Consulting Group? We're #hiring in #Westchester, NY! Click for details:… https://t.co/WKxWa4GXXD
Diane and the great Phyllis Bush at #NPE18Indy https://t.co/usdEj3r2OK
#NPE18Indy with Yevonne Brannon of Public Schools First North Carolina and Lisa Rudley of NYSAPE, leader of Opt Out… https://t.co/6RcqeseAoV
With ⁦@RebeccaGarelli⁩ fabulous leader of Arizona #RedForEd #NPE18Indy https://t.co/KYWZHypJ8t
Love being in New Mexico campaigning for an amazing person and candidate for Governor @RepLujanGrisham .. she will… https://t.co/lrPySqAclj
RT @UFT: John Dewey, philosopher and educator, born this day in 1859. https://t.co/cKZxULnlIR
RT @AFTNM: What a great day to vote early in #NM! Excited that @rweingarten is helping kick off our rally! https://t.co/5YwkPrpVDo
A recent examination of the City Connects program has found that it successfully reduced high school dropout rates.… https://t.co/G0QhLpPgyC
N

In [13]:
#Show LeBron's Timeline
lebron_timeline = api.user_timeline('KingJames')
for tw in lebron_timeline:
    print(tw.text)

RT @BlazePizza: We're fire'd up for the first #LAbron home game tonight at @STAPLESCenter! If you're heading to the game, come grab a speci…
RT @Nike: Just a small-town kid with big-city dreams. #justdoit @KingJames https://t.co/okTQBM8hLI
I will for sure be watching!! 👀 https://t.co/zzTZX8auIR
🙏🏾 https://t.co/tXrQQBivWt
💜💛💜💛💜💛💜💛💜💛💜💛 https://t.co/5i1s53byHz
RT @Nike: Kids from Akron don’t just dream it. They do it. #justdoit https://t.co/Cmj2caP2Mq
RT @beatsbydre: The game will never sound the same. @KingJames #Powerbeats3Wireless https://t.co/Uv1dHFr4OR
That’s dope @Sprite!! 🙏🏾👑 https://t.co/UQExzag1Gt
It’s time everyone! Make your plans and get out and vote. No reason not to. 🙏🏾🗣 https://t.co/ERLjHFGHg6
Super Dope!!! 🔥🔥🔥🔥🔥 Congrats @kylekuzma https://t.co/hB346GJjcF
🤣🤣🤣🤣🤣🤣🤣🤣🤣. It’s ok, we forgive you brother https://t.co/ahE0LR9Jxy
Couple more days until we’re clear for takeoff 🛫! #LakeShow #StriveForGreatness🚀 #KingMe👑 https://t.co/P4aWpY73dX
🔥🔥🔥🔥🔥🔥 🍕 https://t.co/E1LFv1qN2u
Man tha

In [23]:
#Pull All Lebron Tweets available and save to a JSon file
with open('LeBron_Tweets.json', 'w') as f:

    for tweet in tweepy.Cursor(api.user_timeline, screen_name='KingJames').items():
        # process status here
        #print(status.text)
        
        #Write the JSON format to the text file, and add one to the number of tweets we've collected
        f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
        tweetCount += 1

    #Display how many tweets we have collected
    print("Downloaded {0} tweets".format(tweetCount))

Downloaded 3241 tweets


In [126]:
#Examine one tweet from larger JSon file
with open('LeBron_Tweets.json', 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4)) # pretty-print

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Sat Oct 20 18:59:42 +0000 2018",
    "entities": {
        "hashtags": [
            {
                "indices": [
                    46,
                    53
                ],
                "text": "LAbron"
            }
        ],
        "symbols": [],
        "urls": [],
        "user_mentions": [
            {
                "id": 459544233,
                "id_str": "459544233",
                "indices": [
                    3,
                    14
                ],
                "name": "Blaze Pizza",
                "screen_name": "BlazePizza"
            },
            {
                "id": 28725783,
                "id_str": "28725783",
                "indices": [
                    75,
                    89
                ],
                "name": "STAPLES Center",
                "screen_name": "STAPLESCenter"
            }
        ]
    },
    "favorite_count": 0,
    "favorited":

In [101]:
#Put attributes from each Tweet into a DataFrame
lebron_df = pd.DataFrame(columns=['id', 'user', 'date_created', 'retweet', 'retweet_count', 'favorite_count', 'text'])

with open('LeBron_Tweets.json', 'r') as f:
    ind = 0
    for line in f:
        tweet = json.loads(line)
        
        id = tweet['id']
        user = tweet['user']['screen_name']
        created_at = tweet['created_at']
        time_created = datetime.datetime.strptime(created_at, '%a %b %d %H:%M:%S +0000 %Y')
        date_created = time_created.date()
        retweet_count = tweet['retweet_count']
        favorite_count = tweet['favorite_count']
        text = tweet['text']
        retweet = (text[0:2] == 'RT')
        
        tweet_info = pd.DataFrame(data = [[id, user, date_created, retweet, retweet_count, favorite_count, text]], \
                                  index = [ind], \
                                  columns = ['id', 'user', 'date_created', 'retweet', 'retweet_count', 'favorite_count', 'text'])
        lebron_df = lebron_df.append(tweet_info)
        ind += 1

In [102]:
lebron_df.head(5)

id       user date_created retweet retweet_count  \
0  1053722444429615104  KingJames   2018-10-20    True            98   
1  1053722345498529792  KingJames   2018-10-20    True          7955   
2  1053343353260060673  KingJames   2018-10-19   False           454   
3  1053086692318437376  KingJames   2018-10-19   False           954   
4  1053048641575825409  KingJames   2018-10-18   False          3346   

  favorite_count                                               text  
0              0  RT @BlazePizza: We're fire'd up for the first ...  
1              0  RT @Nike: Just a small-town kid with big-city ...  
2           4123  I will for sure be watching!! 👀 https://t.co/z...  
3          11244                         🙏🏾 https://t.co/tXrQQBivWt  
4          29074               💜💛💜💛💜💛💜💛💜💛💜💛 https://t.co/5i1s53byHz

In [124]:
#Keep only tweets from the last year
one_year_ago = datetime.date(2017, 10, 20)
print(one_year_ago)
within_one_year = lebron_df['date_created'] > one_year_ago
lebron_one_year_tweets = lebron_df[within_one_year]

2017-10-20


In [112]:
#Output dataframe to explore
writer = pd.ExcelWriter('output.xlsx')
lebron_one_year_tweets.to_excel(writer,'LeBron')
writer.save()

In [137]:
#Calculate summary statistics on tweets
num_total_tweets = len(lebron_one_year_tweets.index)
num_retweets = lebron_one_year_tweets['retweet'].sum()
num_times_retweeted = lebron_one_year_tweets['retweet_count'].sum()
num_times_favorited = lebron_one_year_tweets['favorite_count'].sum()

In [138]:
#Pull Follower Count
with open('LeBron_Tweets.json', 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    screen_name = tweet['user']['screen_name']
    num_followers = tweet['user']['followers_count']

#print(num_followers)

In [144]:
#Save stats to a player-level dataframe
player_tweet_info = pd.DataFrame(data = [[screen_name, num_total_tweets, num_retweets, num_times_retweeted, num_times_favorited, num_followers]], \
                                  columns = ['screen_name', 'num_total_tweets', 'num_retweets', 'num_times_retweeted', 'num_times_favorited', 'num_followers'])
player_tweet_info.head(5)

screen_name  num_total_tweets  num_retweets  num_times_retweeted  \
0   KingJames               494           106              2883072   

   num_times_favorited  num_followers  
0             13076736       41870432